In [ ]:
# 1. Query Transformation  (질문 변화) - 검색 최적화
# 2. Multi-Query            (다중 질의) - 검색 범위 확대
# 3. Self-RAG               (자기 보정) - 문서 관련성 평가
# 4. Contextual Compressioin (문맥 압축) - 관련 부분만 추출
# 5. Fusion Retrieval       (융합 검색) - 키워드 + 벡터 검색 결합

In [9]:
import os
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')
if not api_key :
    raise ValueError('OPENAI_API_KEY')

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [ ]:
# 문서로드
# 텍스트 분할 - 청킹
# 임베딩 및 VectorDB
# 리트리버
# LLM 설정


In [5]:
# 문서로드
# paths = 'C:/python_src/6.openAI/6'

script_dir = os.getcwd()
docs_path = os.path.join(script_dir)
print(f'docs path : {docs_path}')

loader = DirectoryLoader(
    docs_path,
    glob = '**/*.txt',
    loader_cls = TextLoader,
    loader_kwargs={'encoding':'utf-8'},  # 한국어면 꼭 써줄 것
)
document = loader.load()
print(f'읽은 문서의 수 {len(document)}')

docs path : c:\python_src\6.openAI\6
읽은 문서의 수 5


In [6]:
# 텍스트 분할 - 청킹

text_splitter = RecursiveCharacterTextSplitter(
     chunk_size=100,
     chunk_overlap=50,
     separators = ['\n\n','\n','.',' ',''],
     length_function=len
)

# 스플릿 = 청킹
doc_splits = text_splitter.split_documents(document)
print(f'청킹개수 : {len(doc_splits)}')

청킹개수 : 20


In [ ]:
# 임베딩 및 VectorDB
embedding_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

vectorstroe = Chroma.from_documents(
    documents= doc_splits,
    collection_name='basic_rag_collection',
    embedding=embedding_model
)

In [23]:
# 리트리버

base_retriever = vectorstroe.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':3}
)


In [24]:
# LLM 설정

llm = ChatOpenAI(model = 'gpt-4o-mini',temperature=0)
print(f'setup complete')

setup complete


In [25]:
# 유틸리티 함수 # 프로그램에서 반복적으로 쓰이는 작은 기능을 재사용 가능하게 모아둔 함수
def format_docs(docs):
    '''문서를 문자열로 포멧팅'''
    return '\n\n---\n\n'.join([doc.page_content for doc in docs])

In [ ]:
# 질문 재작성 프롬프트
rewrite_prompt=  ChatPromptTemplate.from_template('''
다음 질문을 검색에 더 적합한 형태로 변환해 주세요.
키워드 중심으로 명확하게 바꿔주세요
변환된 검색어만 출력하세요

원본 질문: {question}
변환된 검색어:
''')

rewrite_chain =  rewrite_prompt | llm | StrOutputParser()

# RAG프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ('system','제공된 문맥을 바탕으로 한국어로 답변하세요'),
    ('human', '문맥:\n{context}\n\n질문:{question}\n\n답변:')
])

def query_transformation(question):
    '''Query Transformation  (질문 변화) - 검색 최적화'''
    print(' 1. Query Transformation  (질문 변화) - 검색 최적화')
    print('사용자 질문을 검색에 최적화된 형태로 변환합니다.\n')

    # 1. 질문 변환
    transformed = rewrite_chain.invoke({'qeustion' : question})
    print(f'원본 질문 : {question}')
    print(f'transformed 질문 : {transformed}')
    
    # 2. 변환된 질문으로 검색
    docs = base_retriever.invoke(transformed)
    context = format_docs(docs) # format_docs 여러 Document 객체를 LLM이 이해할 수 있는 문자열로 합치는 역할

    answer_chain = rag_prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({'context':context, 'question':question})
    return answer, [ os.path.basename(d.metadata.get('source','unknown')) for d in docs ]

test_question = [
    'RAG란 무엇인가요',
    'LangGrraph의 핵심 개념을 설명해주세요']

for q in test_question:
    print(f'Question : {q}')
    answer, sources = query_transformation(q)
    print(f'answer : {answer}  sources : {sources}')


Question : RAG란 무엇인가요
 1. Query Transformation  (질문 변화) - 검색 최적화
사용자 질문을 검색에 최적화된 형태로 변환합니다.

원본 질문 : RAG란 무엇인가요
transformed 질문 : RAG 정의 의미
answer : RAG(검색 증강 생성, Retrieval-Augmented Generation)는 자연어 처리 기술로, 사용자의 질문에 대한 답변을 생성하기 위해 외부 데이터베이스에서 정보를 검색하고 이를 활용하는 방법입니다. RAG는 다음과 같은 장점을 가지고 있습니다:

1. 최신 정보를 반영할 수 있어, 사용자가 원하는 최신 데이터를 제공할 수 있습니다.
2. 환각(Hallucination) 현상을 줄여, 보다 정확하고 신뢰할 수 있는 답변을 생성합니다.
3. 출처를 명시할 수 있어, 제공된 정보의 신뢰성을 높입니다.

RAG의 작동 원리는 다음과 같습니다:

1. 사용자의 질문을 임베딩 벡터로 변환합니다.
2. 벡터 데이터베이스에서 유사한 문서를 검색합니다.
3. 검색된 문서를 컨텍스트로 사용하여 대형 언어 모델(LLM)이 답변을 생성합니다.

이러한 과정을 통해 RAG는 보다 정확하고 유용한 정보를 제공할 수 있습니다.  sources : ['rag_concept.txt', 'rag_concept.txt', 'rag_concept.txt']
Question : LangGrraph의 핵심 개념을 설명해주세요
 1. Query Transformation  (질문 변화) - 검색 최적화
사용자 질문을 검색에 최적화된 형태로 변환합니다.

원본 질문 : LangGrraph의 핵심 개념을 설명해주세요
transformed 질문 : LangGraph 핵심 개념 설명
answer : LangGraph의 핵심 개념은 다음과 같습니다:

1. **State(상태)**: 에이전트의 현재 상태를 나타내는 데이터 구조로, 에이전트가 수행하는 작업이나 프로세스의 진행 상황을 추적하는 데 사용됩니다.

2. **Node(노드)*

In [ ]:
# 2. Multi-Query            (다중 질의) - 검색 범위 확대    # 쿼리는 검색 엔진이나 벡터 DB에 넣는 검색어를 말함
# 다중 쿼리 생성 프롬프트
multi_query_prompt = ChatPromptTemplate.from_template('''
다음질문에 대해 3가지 다른 관점의 검색 쿼리를 생성하세요.
각쿼리는 세 줄로 구분하여 출력하세요        
번호나 설명 없이 쿼리만 출력하세요
원본질문 : {question}
다른 관점의 쿼리들 ''')
                                # 세 줄로 구분하여, 3가지 다른 관점의~~ => 다중 쿼리를 만드는 프롬프트
# lag chain 구성 LCEL
multi_query_chain = multi_query_prompt | llm | StrOutputParser()

def multi_query_rag(question):
    '''다중 쿼리로 검색해서 결과 통일'''
    # multi_query_chain 실행해서 결과출력
    queries_text = multi_query_chain.invoke( {'question': question} )
    print(queries_text)
    queries = [q.strip() for q in queries_text.strip().split('\n')  if q.strip()]   # 공백제거, 줄띄움을 기준으로 나눠준다
    # 각 쿼리(질문)으로 검색하고 결과를 통합(중복제거)
    all_docs = [] # 각각의 답을 리스트에 담는다
    seen_contents = set()
    for query in queries:   # 각 3개의 쿼리로 검색 실행
        docs = base_retriever.invoke(query)  # 각 3개의 쿼리들을 리트리버에서 찾기. 각각 따로 작동한다
        for doc in docs: 
            if doc.page_content not in seen_contents:   # document에는 page_content(속성에 본문 내용),metadata(속성출처) 의 두가지 정보를 가진다/ page_content는 여기에서 문서 실제 내용을 불러오는 것
                seen_contents.add(doc.page_content) # 중복된 내용 체크
                all_docs.append(doc)                # 중복을 제외하고 저장

    print(f'검색된 문서의 개수 : {len(all_docs)}')
    # 리트리버 답변 생성 상위 3개만 사용
    context = format_docs(all_docs[:3])
    answer_chain = rag_prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({'context' : context, 'question':question})    # {context} = 위에서 만든 3개 문서 합친 내용
    return answer, [ os.path.basename(d.metadata.get('source','unknown')) for d in all_docs ]
     
test = [
    'LangChain 시작하는 방법'
]
for q in test:
    print(f'question : {q}')
    answer, sources = multi_query_rag(q)
    print(f'answer : {answer}')
    print(f'answer : {sources}')



question : LangChain 시작하는 방법
LangChain 사용법 튜토리얼  

LangChain 시작하기 위한 기본 가이드  

LangChain 설치 및 설정 방법  
검색된 문서의 개수 : 3
answer : LangChain을 시작하는 방법은 다음과 같습니다:

1. **환경 설정**: Python이 설치된 환경을 준비합니다. 필요한 패키지를 설치하기 위해 `pip`를 사용합니다. 예를 들어, LangChain을 설치하려면 다음 명령어를 실행합니다:
   ```bash
   pip install langchain
   ```

2. **모델 선택**: LangChain은 다양한 LLM 제공자와 통합되어 있습니다. OpenAI, Anthropic 등에서 제공하는 모델 중 하나를 선택하고 API 키를 준비합니다.

3. **프롬프트 설정**: 프롬프트 템플릿을 관리하고 최적화하기 위해 필요한 프롬프트를 정의합니다. 이를 통해 모델이 원하는 방식으로 응답하도록 유도할 수 있습니다.

4. **애플리케이션 개발**: LangChain의 구성 요소를 활용하여 애플리케이션을 개발합니다. 모델과 프롬프트를 결합하여 원하는 기능을 구현합니다.

5. **테스트 및 배포**: 개발한 애플리케이션을 테스트하고, 필요에 따라 수정한 후 배포합니다.

이 과정을 통해 LangChain을 활용한 애플리케이션 개발을 시작할 수 있습니다. 추가적인 문서나 튜토리얼을 참고하면 더 많은 정보를 얻을 수 있습니다.
answer : ['langchain_intro.txt', 'langchain_intro.txt', 'langgraph_intro.txt']


In [ ]:
# 3. Self-RAG               (자기 보정) - 문서 관련성 평가

print(f'3.Self-RAG')
print(f'검색된 문서의 관련성을 평가하여 필터링합니다.\n')

# 프롬프트 
check_prompt = ChatPromptTemplate.from_template(''' 
다음 문서가 다음 질문에 관련이 있는지 평가하세요
'YES'또는 'NO'로만 대답하세요

문서: {document}
질문: {question}
관련성: 
 ''')
# LCEL 체인 구성
check_prompt_chain = check_prompt | llm | StrOutputParser()

def filter_relevant_docs(docs, question):   # docs → 리트리버가 검색해서 가져온 문서 리스트/question 사용자가 입력한 질문 문자열
    '''관련 있는 문서만 필터링'''
    relevant = []       # 필터링된 문서를 담을 리스트
    for doc in docs :   # docs 리스트를 하나씩 가져옴/ # doc → Document 객체
        result = check_prompt_chain.invoke({    # llm은 
        'document': doc.page_content,
        'question' : question})
        is_relevant = 'yes' in result.lower()   # 문자열에 yes라는 글자가 있으면” 관련 있다고 판단
        print(f"    -{doc.page_content[:50]}.... : {"Relevent" if is_relevant else "Not Relevent"}")
        if is_relevant:
            relevant.append(doc)
    return relevant


# 1) 문서를 검색(리트리버를 이용해서)
question = "RAG의 장점은 무엇인가요?"
docs = base_retriever.invoke(question)
print(f"리트리버가 찾은 문서 수 : {len(docs)}개")

# 2) 관련성 평가
relevant_docs = filter_relevant_docs(docs, question)
print(f"관련 문서 수 : {len(relevant_docs)}개")

if not relevant_docs:
    raise ValueError('관련있는 문서가 없어서 답변을 종료합니다.다른 질문을 입력하세요')


def format_docs(docs):
    '''문서를 문자열로 포멧팅'''
    return '\n\n---\n\n'.join([ doc.page_content for doc in docs ])

context = format_docs(relevant_docs)

#답변 생성
#RAG프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ('system','제공된 문맥을 바탕으로 한국어로 답변하세요'),
    ('human', '문맥:\n{context}\n\n질문:{question}\n\n답변:')
])
answer_chain = rag_prompt | llm | StrOutputParser()
answer = rag_prompt.invoke({'context' : context, 'question' : question})
print(f' 답변 : {answer}')
sources = [ os.path.basename(doc.metadata.get('source',"")) for doc in relevant_docs]
print(f' 근거 : {sources}')


3.Self-RAG
검색된 문서의 관련성을 평가하여 필터링합니다.

리트리버가 찾은 문서 수 : 3개
    -RAG의 장점:
- 최신 정보를 반영할 수 있음
- 환각(Hallucination) 감소
.... : Relevent
    -RAG의 작동 원리:
1. 사용자 질문을 임베딩 벡터로 변환
2. 벡터 데이터베이스에서 유.... : Not Relevent
    -RAG (Retrieval-Augmented Generation)는 검색 증강 생성 기술입.... : Relevent
관련 문서 수 : 2개
 답변 : messages=[SystemMessage(content='제공된 문맥을 바탕으로 한국어로 답변하세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='문맥:\nRAG의 장점:\n- 최신 정보를 반영할 수 있음\n- 환각(Hallucination) 감소\n- 출처를 명시할 수 있음\n\n---\n\nRAG (Retrieval-Augmented Generation)는 검색 증강 생성 기술입니다.\n\n질문:RAG의 장점은 무엇인가요?\n\n답변:', additional_kwargs={}, response_metadata={})]
 근거 : ['rag_concept.txt', 'rag_concept.txt']


In [90]:
# 4. Contextual Compressioin (문맥 압축) - 관련 부분만 추출

question = 'VectorDB의 종류를 알려주세요'

# 1. 문맥압축 프롬프트를 실행
compress_prompt = ChatPromptTemplate.from_template(
'''
다음 문서가 질문에 관련이 있는지 평가하세요.
관련 없는 부분은 제외하고, 관련 있는 내용만 그대로 출력하세요.
관련 내용이 없으면 '관련 없음'이라고 출력하세요

문서: {document}
질문: {question}
관련성:                                                                                                                                    
'''

)

docs = base_retriever.invoke(question)
compressed = []
sources = []
for doc in docs: 
    document = doc.page_content
    compress_chain = compress_prompt | llm | StrOutputParser()
    compress_result = compress_chain.invoke({"question":question, "document":document})

    if '관련없음' not in compress_result:
        compressed.append(compress_result)
        sources.append(os.path.basename(doc.metadata.get('source',"")))
context = '\n\n---\n\n'.join(compressed)

# 최종답변
# RAG프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ('system','제공된 문맥을 바탕으로 한국어로 답변하세요'),
    ('human', '문맥:\n{context}\n\n질문:{question}\n\n답변:')
])
rag_prompt_chain = rag_prompt | llm | StrOutputParser()
result = rag_prompt_chain.invoke({'context': context, 'question' : question})
print(result,sources)


VectorDB의 종류에는 여러 가지가 있습니다. 대표적인 예로는 다음과 같은 것들이 있습니다:

1. **ChromaDB**: 로컬 개발에 적합한 오픈소스 솔루션으로, 개발자들이 쉽게 사용할 수 있도록 설계되었습니다.
2. **Pinecone**: 완전 관리형 클라우드 서비스로, 대규모 데이터 처리와 빠른 검색 성능을 제공합니다.
3. **Faiss**: Facebook에서 개발한 라이브러리로, 대규모 벡터 검색을 위한 효율적인 방법을 제공합니다.
4. **Annoy**: Spotify에서 개발한 라이브러리로, 근사 최근접 이웃 검색을 위한 효율적인 구조를 가지고 있습니다.
5. **Milvus**: 오픈소스 벡터 데이터베이스로, 대규모 데이터셋을 처리하고 실시간 검색을 지원합니다.

이 외에도 다양한 VectorDB 솔루션이 있으며, 각기 다른 용도와 기능을 가지고 있습니다. ['vectordb_intro.txt', 'vectordb_intro.txt', 'langchain_intro.txt']


In [100]:
# 5. Fusion Retrieval       (융합 검색) - 키워드 + 벡터 검색 결합

# 1. 개별 검색 결과 비교

# 2. 융합 결과로 답변 생성

from langchain_community.retrievers import BM25Retriever
# BM25 리트리버   : 키워드 기반
# Vector 리트리버 : 의미기반
bm25_retriever = BM25Retriever.from_documents(doc_splits)
bm25_retriever.k = 3

question = 'VectorDB의 종류를 알려주세요'
# 벡터 검색
vector_docs = base_retriever.invoke(question)
#bm25 검색
bm25_docs = bm25_retriever.invoke(question)
fusion_scores = {}
# 벡터 검색 결과 점수화
for rank, doc in enumerate(vector_docs):
    doc_key = doc.page_content[:50]
    score =1/(60+rank)
    fusion_scores[doc_key] = fusion_scores.get(doc_key,0) + score
# bm25 검색 결과 점수화
for rank, doc in enumerate(bm25_docs):
    doc_key = doc.page_content[:50]
    1/(60+rank)
    fusion_scores[doc_key] = fusion_scores.get(doc_key,0) + score
# 점수로 정렬
sorted_docs = sorted(
    fusion_scores.items(),key=lambda x : x[1], reverse = True
)
print('fusion docs 결과 상위 3개 : {sorted_docs[:3]}')
docs = []
for doc,score in sorted_docs[:3]:
    docs.append(doc)

inputs = '\n\n---\n\n'.join(docs)

rag_prompt_chain = rag_prompt | llm | StrOutputParser()
result = rag_prompt_chain.invoke({'context' : inputs, 'question' : question})
print(result)


fusion docs 결과 상위 3개 : {sorted_docs[:3]}
VectorDB의 주요 종류는 다음과 같습니다:

1. **ChromaDB**: 로컬 개발에 적합한 오픈소스 솔루션.
2. **Pinecone**: 클라우드 기반의 벡터 데이터베이스로, 확장성과 성능이 뛰어남.
3. **Weaviate**: 그래프 기반의 벡터 데이터베이스로, 스키마리스 구조를 지원.
4. **Milvus**: 대규모 벡터 데이터를 처리할 수 있는 오픈소스 솔루션.
5. **Faiss**: Facebook에서 개발한 벡터 검색 라이브러리로, 대량의 벡터를 효율적으로 검색할 수 있음.

이 외에도 다양한 VectorDB 솔루션이 있으며, 각 솔루션은 특정 용도와 요구 사항에 맞춰 선택할 수 있습니다.
